In [15]:
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from imblearn.combine import SMOTETomek
import MachineLearning as ML
import PrepareData

import importlib



In [16]:
importlib.reload(PrepareData)

df=PrepareData.get_data()

In [17]:
importlib.reload(ML)

df=df.dropna()

y=df['nextday_rainfall']
X=df.drop('nextday_rainfall', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, stratify=y)

X, y = SMOTETomek().fit_resample(X_train, y_train)

model=ML.TreeGradientBoosting(X,y)

print(model)

y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)

f1_sc = f1_score(y_test.values.ravel(), y_pred.ravel())
print(f1_sc)

LGBMClassifier()
[[501 266]
 [257 985]]
0.7902125952667469
